In [129]:
from sage.all import *
import sys
import math

N = 11
p = 3
q = 2^5
#d =
Zx.<x> = ZZ[]
f = -1 + x + x^2 -x^4 +x^6 + x^9 - x^10
g = -1 + x^2 + x^3 + x^5 - x^8 - x^10
m=-1+x^3-x^4-x^8+x^9+x^10

In [2]:
def balancedmod(f,q):
    g = list(((f[i] + q//2) % q) - q//2 for i in range(N))
    return Zx(g)

In [12]:
def multiply(f,g):
    return (f * g) % (x^N-1)

In [4]:
def invertmodprime(f,p):
    T = Zx.change_ring(Integers(p)).quotient(x^N-1)
    return Zx(lift(1 / T(f))) 

In [133]:
def test_invertmodprime():
    #f = -1 + x + x^2 -x^4 +x^6 + x^9 - x^10
    #p=3
    f_p=invertmodprime(f,p)
    print("f_p =",f_p)
test_invertmodprime()


f_p = 2*x^9 + x^8 + 2*x^7 + x^5 + 2*x^4 + 2*x^3 + 2*x + 1


In [118]:
def invertmodpowerof2(f,q):
    a=2
    b=invertmodprime(f,2)
    while a<q:
        a=a^2
        b=multiply(b,(2-multiply(f,b)))
        Za.<z>=PolynomialRing(Integers(a))
        f1=Zx(Za(b))
    #Zq.<x>=PolynomialRing(Integers(q))
    return b

In [123]:
def test_invertmodpowerof2():
    f_q=invertmodpowerof2(f,q)
    Zq.<x>=PolynomialRing(Integers(q))
    print("f_q =", Zx(Zq(f_q)))
test_invertmodpowerof2()

f_q = 30*x^10 + 18*x^9 + 20*x^8 + 22*x^7 + 16*x^6 + 15*x^5 + 4*x^4 + 16*x^3 + 6*x^2 + 9*x + 5


In [77]:
def validate_params():
    if q > p and gcd(p,q) == 1:
        return True
    return False

In [71]:
def generate_polynomial(d):

    assert d <= N
    result = N*[0]
    for j in range(d):  
        while True:
            r = randrange(N)    
            if not result[r]: break
        result[r] = 1-2*randrange(2) 
    return Zx(result)



In [72]:
def generate_keys():
    if validate_params():
        while True:
            try:
                f = generate_polynomial(d)
                g = generate_polynomial(d)
                f_q = invertmodpowerof2(f,q)
                f_p = invertmodprime(f,p)  
                break
        
            except:
                pass 
        public_key = balancedmod(p * multiply(f_q,g),q)
        secret_key = f,f_p
        return public_key,secret_key
    else:
        print("Provided params are not correct. q and p should be co-prime, q should be a power of 2 considerably larger than p and p should be prime.")



In [78]:
def generate_message():
    result = list(randrange(3) - 1 for j in range(N))
    return Zx(result)

In [86]:
def encrypt(message, public_key):   
    #r = generate_polynomial(d)
    r=-1+x^2+x^3+x^4-x^5-x^7
    #m=-1+x^3-x^4-x^8+x^9-x^10
    return balancedmod(multiply(public_key,r) + message,q)

In [130]:
def test_encrypt():
    h=balancedmod(multiply(g,p*invertmodpowerof2(f,q)),q)
    e=encrypt(m, h)
    print("h =", h)
    Zq.<x>=PolynomialRing(Integers(q))
    print("e =",Zx(Zq(e)))
test_encrypt()

h = -16*x^10 - 13*x^9 + 12*x^8 - 13*x^7 + 15*x^6 - 8*x^5 + 12*x^4 - 12*x^3 - 10*x^2 - 7*x + 8
e = 19*x^10 + 6*x^9 + 25*x^8 + 7*x^7 + 30*x^6 + 16*x^5 + 14*x^4 + 24*x^3 + 26*x^2 + 11*x + 14


In [147]:
def decrypt(encrypted_message, secret_key):   
    f,f_p = secret_key
    a = balancedmod(multiply(encrypted_message,f),q)
    return balancedmod(multiply(a,f_p),p)


In [153]:
def test_decrypt():
    h=balancedmod(multiply(g,p*invertmodpowerof2(f,q)),q)
    e=encrypt(m, h)
    secret_key=f,invertmodprime(f,p)
    a = balancedmod(multiply(e,f),q)
    print("a =", a)
    b=balancedmod(a,p)
    print("b =", b)
    c=decrypt(e,secret_key)
    print("c =", c)
test_decrypt()
    

a = -7*x^10 - 3*x^9 + 5*x^8 + 7*x^7 + 6*x^6 + 7*x^5 + 10*x^4 - 11*x^3 - 10*x^2 - 7*x + 3
b = -x^10 - x^8 + x^7 + x^5 + x^4 + x^3 - x^2 - x
c = x^10 + x^9 - x^8 - x^4 + x^3 - 1
